In [2]:
# 예시 데이터

import pandas as pd
from io import StringIO

csv_data = \
'''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''

df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [3]:
# pandas의 isnull: null이면 true, 아니면 false인 dataframe을 반환하는 함수
# isnull().sum()을 통해 전체 null 개수를 확인할 수 있다.
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

In [4]:
# 데이터 전처리는 pandas dataframe으로 하는게 편리
# scikit-learn에 넣을때는 numpy로 넣어주어야 함
# .values를 통해 넘파이 배열로 변경 가능
df.values

array([[ 1.,  2.,  3.,  4.],
       [ 5.,  6., nan,  8.],
       [10., 11., 12., nan]])

In [15]:
# 누락값을 다루는 방법 (1) 누락값 제거

# dropna(axis): null이 있는 샘플 행이나 열 삭제
## .dropna(axis = 0): 누락된 값이 있는 행 삭제
print(df.dropna(axis=0))
print('=======')

## .dropna(axis = 1): 누락된 값이 있는 열 삭제
print(df.dropna(axis=1))
print('=======')

## .dropna(how = 'all'): 모든 열이 nan일때만 행 삭제
print(df.dropna(how = 'all'))
print('=======')

## .dropna(thresh): 실수 값이 thresh보다 작은 행 삭제
print(df.dropna(thresh = 4))
print('=======')

## .dropna(subset): 특정 열에 nan이 있는 행만 삭제
print(df.dropna(subset = ['C', 'D']))

     A    B    C    D
0  1.0  2.0  3.0  4.0
      A     B
0   1.0   2.0
1   5.0   6.0
2  10.0  11.0
      A     B     C    D
0   1.0   2.0   3.0  4.0
1   5.0   6.0   NaN  8.0
2  10.0  11.0  12.0  NaN
     A    B    C    D
0  1.0  2.0  3.0  4.0
     A    B    C    D
0  1.0  2.0  3.0  4.0


In [28]:
# 누락값을 다루는 방법 (2) 누락값 보강(interpolation)

# simpleimputer: 누락된 값을 평균, 최빈값 등으로 대체해줌
## strategy = 'mean'(평균), 'median'(중간값), 'most_frequent'(최빈값)
## strategy = 'constant', fill_value = @ (nan을 @로 채워줌)

# 누락된 값을 열의 평균으로 대체합니다.
import numpy as np
from sklearn.impute import SimpleImputer

imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

In [30]:
# SimpleImputer는 열 값의 특성으로 데이터를 바꿔줌
## 만약 행 값으로 바꾸고 싶은 경우, 입력 행렬의 행과 열을 바꿔 계산한 후 다시 행렬을 변환함
from sklearn.preprocessing import FunctionTransformer
ftr_simr = FunctionTransformer(lambda X: imr.fit_transform(X.T).T, validate=False)
imputed_data = ftr_simr.fit_transform(df.values)
imputed_data

array([[ 1.        ,  2.        ,  3.        ,  4.        ],
       [ 5.        ,  6.        ,  6.33333333,  8.        ],
       [10.        , 11.        , 12.        , 11.        ]])

In [31]:
# transformer와 estimator

# transformer: fit, transform을 사용하여 데이터를 학습하고 변환함
## 변환하려는 데이터 배열은 학습에서 사용한 데이터의 개수와 같아야 함

# estimator: fit, predict(transform을 가질수도 있음)를 통해 데이터를 예측

In [59]:
# 범주형 데이터를 다루는 방법

# 예제 데이터셋
import pandas as pd

df = pd.DataFrame([['green', 'M', 10.1, 'class1'],
                   ['red', 'L', 13.5, 'class2'],
                   ['blue', 'XL', 15.3, 'class1']])

df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


In [60]:
# 클래스 레이블 인코딩 (1)
## scikit-learn의 대부분 분류기는 알아서 정수로 클래스를 분류해주기도 하지만, 직접 하는게 미연의 사고 방지에 좋음
## enumerate, dictionary 사용해서 class를 정수로 매핑
class_mapping = {label: idx for idx, label in enumerate(np.unique(df['classlabel']))}
print(class_mapping)
print('=======')

# 매핑 딕셔너리를 사용하여 클래스를 정수로 변환
df['classlabel'] = df['classlabel'].map(class_mapping)
print(df)
print('=======')

# 매핑 딕셔너리를 뒤집어서 다시 원본 문자열로 변환
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
print(df)

{'class1': 0, 'class2': 1}
   color size  price  classlabel
0  green    M   10.1           0
1    red    L   13.5           1
2   blue   XL   15.3           0
   color size  price classlabel
0  green    M   10.1     class1
1    red    L   13.5     class2
2   blue   XL   15.3     class1


In [61]:
# 클래스 레이블 인코딩 (2)
## sklearn LabelEncoder 사용해서 간단하게 변환 가능
from sklearn.preprocessing import LabelEncoder

class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
print(y)

## inverse_transform으로 뒤집을수도 있음
inv_y = class_le.inverse_transform(y)
print(inv_y)


[0 1 0]
['class1' 'class2' 'class1']


In [67]:
# one-hot encoding: 클래스마다 새로운 더미값을 만듬
# ColumnTransformer: 데이터프레임 열마다 다른 변환을 적용하도록 도와줌
## transformer: 이름, 변환기, 변환할 열의 리스트로 구성
X = df[['color', 'size', 'price']].values

color_le = LabelEncoder()
X[:, 0] = color_le.fit_transform(X[:, 0])

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

oh_enc = OneHotEncoder(categories='auto')
col_trans = ColumnTransformer([('oh_enc', oh_enc, [0])], remainder='passthrough')
col_trans.fit_transform(X)

array([[0.0, 1.0, 0.0, 'M', 10.1],
       [0.0, 0.0, 1.0, 'L', 13.5],
       [1.0, 0.0, 0.0, 'XL', 15.3]], dtype=object)

In [69]:
# get_dummies: 문자열 열을 모두 더미로 변환

pd.get_dummies(df[['price', 'color', 'size']])

,price,color_blue,color_green,color_red,size_L,size_M,size_XL
0,10.1,0,1,0,0,1,0
1,13.5,0,0,1,1,0,0
2,15.3,1,0,0,0,0,1


In [72]:
# get_dummies : columns = ['@'] 를 지정해서 특정 컬럼만 더미로 바꾸는것도 가능
pd.get_dummies(df[['price', 'color', 'size']], columns=['size'])

,price,color,size_L,size_M,size_XL
0,10.1,green,0,1,0
1,13.5,red,1,0,0
2,15.3,blue,0,0,1


In [75]:
# 다중공선성(multicollinearity) 문제
## 변수간의 상관관계가 높을 경우, 특정 ml 알고리즘에서는 문제가 될 수 있음
## 변수간 상관관계를 줄이기 위해 one-hot encoding에서는 하나를 제거하는게 좋음

# get_dummies: drop_first를 사용해서 하나 삭제가능
pd.get_dummies(df[['price', 'color', 'size']], drop_first=True)

,price,color_green,color_red,size_M,size_XL
0,10.1,1,0,1,0
1,13.5,0,1,0,0
2,15.3,0,0,0,1
